In [18]:
import psycopg2
import sys
from sqlalchemy import create_engine
import pandas as pd
from setting import *
from lxml import html
import hanja
from nltk.tokenize import RegexpTokenizer
def init():
    if sys.platform == "win32":
        conn = psycopg2.connect(database=DB, user=USER, password=PASSWORD, host='localhost', port=PORT)
        cur = conn.cursor()
    else:
        print("The OS of this machine is " + sys.platform + ". Connecting to remote server...")
        # host = '143.248.109.99'
        conn = psycopg2.connect(database=DB, user=USER, password=PASSWORD, host=HOST, port=PORT)
        cur = conn.cursor()
    return conn, cur

engine = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(USER, PASSWORD, HOST,PORT,DB))
con, cur = init()

The OS of this machine is darwin. Connecting to remote server...


In [2]:
sillok = pd.read_sql_table('sillok', con=engine)

In [140]:
import re
specials = set(list(re.sub(r'[^\x00-\x7F]+','',''.join(sillok['chinese']))))

In [161]:
specials = {'○', '。', ',','"', '?','/','、','而', 'ㆍ'}

In [255]:
dates = pd.read_sql_table('dates',con=engine)
dates = {k:v for k, v in zip(dates['lunardate'], dates['solardate'])}
sillok['solardate'] = sillok['lunardate'].map(dates)

In [272]:
con, cur = init()
def find_gwanjik(x):
    p = x['paragraph']
    sid = x['sid']
    solardate = x['solardate']
    para = ''
    tglist = re.findall('<index.*?>.+?</index>','<p>'+p+'</p>')
    nmlist = re.findall('<index.*?>(.+?)</index>','<p>'+p+'</p>')
    
    for i, j in enumerate(html.fromstring('<p>'+p+'</p>').xpath('//text()')):
        if j in nmlist:
            s = tglist[nmlist.index(j)]
            tglist.pop(nmlist.index(j))
            nmlist.pop(nmlist.index(j))
        else:
            s = '<c>'+j+'</c>'
        para += s
    para = '<p>'+para+'</p>'
    tree = etree.parse(StringIO(para))
    root = tree.getroot()
    refs = root.xpath('index/@ref')
    for ref in set(refs):
        gwanjik = [''.join([s for s in list(i.text) if s not in specials]).split(' ')[-1] for i in root.xpath('index[@ref="'+ref+'"]/preceding::c[1]')][0]
        officelevel = hanja.translate(gwanjik,mode='substitution')+'('+gwanjik+')' if gwanjik != '' else None 
        
        cur.execute('''
    insert into sillok_career_new (spid, officelevel, sid, solardate)
    values ((%s), (%s), (%s), (%s))
    ''', (ref, officelevel, sid, solardate))
    

sillok.apply(find_gwanjik, axis=1)
con.commit()
con.close()


The OS of this machine is darwin. Connecting to remote server...


In [11]:
[i.text for i in html.fromstring(sillok['paragraph'][0]).xpath('//index[@type="이름"]')]

['李鎬俊', '趙寅燮', '鄭謙植', '金宗泰', '李應辰', '沈承澤', '嚴錫鼎']

In [18]:
[i for i in html.fromstring(sillok['paragraph'][4]).xpath('//@ref')]

['M_0012208', 'M_0004893', 'M_0058630']

In [21]:
[i for i in html.fromstring(sillok['paragraph'][0]).xpath('text()')]

['陳賀時差備官以下, 施賞有差。 禮房承旨',
 '、對擧承旨',
 '、宣敎官',
 '、左通禮',
 '、右通禮',
 ', 竝加資。 都承旨',
 '、戶曹參判',
 ', 百官加親授。 ']

In [35]:
[i.text for i in txt.xpath('preceding-sibling::index[@ref="M_0003229"]')]

[]

In [41]:
from lxml import etree
from io import StringIO, BytesIO

In [37]:
parser = etree.XMLParser()

In [46]:
tree = etree.parse(StringIO('<p>'+sillok['paragraph'][0]+'</p>'))
root = tree.getroot()

In [56]:
print(etree.tostring(tree.getroot(), pretty_print=True, method='html'))

b'<p>&#38515;&#36032;&#26178;&#24046;&#20633;&#23448;&#20197;&#19979;, &#26045;&#36062;&#26377;&#24046;&#12290; &#31150;&#25151;&#25215;&#26088;<index num="1953057_0" ref="M_0005439" sort="K" type="&#51060;&#47492;">&#26446;&#37804;&#20426;</index>&#12289;&#23565;&#25831;&#25215;&#26088;<index num="1953058_0" ref="M_0026863" sort="K" type="&#51060;&#47492;">&#36249;&#23493;&#29166;</index>&#12289;&#23459;&#25934;&#23448;<index num="1953059_0" ref="M_0027023" sort="K" type="&#51060;&#47492;">&#37165;&#35609;&#26893;</index>&#12289;&#24038;&#36890;&#31150;<index num="1953060_0" ref="M_0026297" sort="K" type="&#51060;&#47492;">&#37329;&#23447;&#27888;</index>&#12289;&#21491;&#36890;&#31150;<index num="1953061_0" ref="M_0004921" sort="K" type="&#51060;&#47492;">&#26446;&#25033;&#36784;</index>, &#31453;&#21152;&#36039;&#12290; &#37117;&#25215;&#26088;<index num="1953062_0" ref="M_0003229" sort="K" type="&#51060;&#47492;">&#27784;&#25215;&#28580;</index>&#12289;&#25142;&#26361;&#21443;&#210

In [126]:
res = set()
for i in sillok['chinese'].str.replace(r'[^\x00-\x7F]+', '').apply(lambda x: set(x.split())).index:


0                                   {,}
1                              {,, /:,}
2                     {",, ,, "", :, "}
3                         {:, ",, ", ,}
4                         {:, ",, ", ,}
                      ...              
384274             {;, ?", ",, ,, :, "}
384275                   {:, ",, ?", ,}
384276                               {}
384277                          {,, /,}
384278    {"?", ?, ,, ",, ":, /,, :, "}
Name: chinese, Length: 384279, dtype: object

In [130]:
'test'

['test']

In [193]:
p = '''<p>○乙丑/上御別殿。 領事<index num="1000512_0" ref="M_0003630" sort="K" type="이름">柳成龍</index>、知事<index num="1000513_0" ref="M_0061334" sort="K" type="이름">洪進</index>、特進官<index num="1000514_0" ref="M_0049182" sort="K" type="이름">申磼</index>ㆍ宋言愼、參贊官<index num="1000515_0" ref="M_0006601" sort="K" type="이름">洪世恭</index>ㆍ<index num="1000516_0" ref="M_0005431" sort="K" type="이름">李好閔</index>、執義<index num="1000517_0" ref="M_0001736" sort="K" type="이름">金時獻</index>、司諫<index num="1000518_0" ref="M_0002037" sort="K" type="이름">金弘微</index>、檢討官<index num="1000519_0" sort="K" type="이름">鄭㷤</index>、記事官<index num="1000520_0" ref="M_0056678" sort="K" type="이름">趙濈</index>ㆍ<index num="1000521_0" ref="M_0000861" sort="K" type="이름">張晩</index>ㆍ<index num="1000522_0" ref="M_0003584" sort="K" type="이름">柳慶宗</index>入侍, 上講<index num="1000523_0" sort="K" type="이름">周易</index>。 <index num="1000524_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "自上欲爲展謁太廟, 人皆以爲感激, 但列聖座次, 當初倉卒不能講定。 頃者<index num="1000525_0" ref="M_0061334" sort="K" type="이름">洪進</index>奉審時, 入而見之, 最爲未安。 不得已先定座次, 然後親祭可矣。" <index num="1000526_0" ref="M_0061334" sort="K" type="이름">洪進</index>曰: "臣與<index num="1000527_0" ref="M_0003630" sort="K" type="이름">成龍</index>奉審, 以圖形進入矣。 大槪十室, 以西爲上, 而其室之制不長, 不能一行而爲座次, 故回坐於越邊, 所見未安故敢達。" 上曰: "禮曹與大臣議爲。" <index num="1000528_0" ref="M_0061334" sort="K" type="이름">洪進</index>曰: "小臣之意, 以爲如昭穆之制可也。 連作一室於其外, 則可坐如昭穆。" <index num="1000529_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "今者日寒, 親祭前, 恐不可爲也。" 上曰: "今此<index num="1000530_0" sort="J" type="지명">倭賊</index>不退, 不擊則我國不可保矣。 設使<index num="1000531_0" sort="J" type="지명">倭</index>雖撤退, 後必更來, 我國人情, 謂其一去, 則不復出來云, 可笑。 明春兇謀, 不可測矣。" 上曰: "<index num="1000532_0" ref="M_0032544" sort="A" type="이름">孫經略</index>爲我國題本云, 卿見之乎?" <index num="1000533_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "不見矣。" 上曰: "大臣不得見乎?" <index num="1000534_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "小臣連有病故, 不得來見矣。" 上曰: "承旨<annotation type="원주">                 <noteContent>【<index num="1000535_0" ref="M_0006601" sort="K" type="이름">洪世恭</index>。】</noteContent>               </annotation>亦見乎?" <index num="1000536_0" ref="M_0006601" sort="K" type="이름">世恭</index>曰: "暫見矣。" 上曰: "其意以爲, 中原、我國, 形勢相倚, <index num="1000537_0" sort="J" type="지명">倭賊</index>之來, 若先保<index num="1000538_0" sort="K" type="지명">朝鮮</index>, 則中原可無憂, 而<index num="1000539_0" sort="K" type="지명">中</index>朝若盡力防賊, 則<index num="1000540_0" sort="K" type="지명">朝鮮</index>亦必有德色矣。 軍糧不足, 則<index num="1000541_0" sort="M" type="지명">金</index>、<index num="1000542_0" sort="K" type="지명">覆</index>、<index num="1000543_0" sort="K" type="지명">海</index>、<index num="1000544_0" sort="K" type="지명">蓋</index>               <annotation type="원주">                 <noteContent>【四州名。】</noteContent>               </annotation>之粟, 移於<index num="1000545_0" sort="K" type="지명">平壤</index>, <index num="1000546_0" sort="M" type="지명">遼東</index>之穀, 移於<index num="1000547_0" sort="K" type="지명">義州</index>云云。 此必有守<index num="1000548_0" sort="K" type="지명">朝鮮</index>之意。 兵部奉聖旨, 已定其軍糧之數云矣。" <index num="1000549_0" ref="M_0049182" sort="K" type="이름">申磼</index>曰: "我國鍊兵, 而片箭最爲長技。 頃見官射, 則武人十人, 無一人射片箭者, 極爲可慮。 如欲鍊兵, 則片箭之技, 當爲培養可矣。" 上曰: "<index num="1000550_0" sort="J" type="지명">倭</index>使出來, 何以爲之?" <index num="1000551_0" ref="M_0005431" sort="K" type="이름">好閔</index>曰: "奏文請禁約使。 若然則可制矣。" 上曰: "禁約使能縶<index num="1000552_0" sort="J" type="지명">倭</index>之手足乎? 其號令不能行矣。" <index num="1000553_0" ref="M_0005431" sort="K" type="이름">好閔</index>曰: "禁約事, 不可不爲於奏文中也。 上曰: "渠之兇謀, 不可測也。 若擧窟穴出來, 直向中原, 則不可說也。" <index num="1000554_0" ref="M_0005431" sort="K" type="이름">好閔</index>曰: "渠雖無此意, 在我之備, 不可忽也。" 上曰: "當今之務, 莫如收用賢才。 其外無他策。" <index num="1000555_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "小臣當此時, 不可堪任, 而冒據至此, 晝夜憂悶而已。 大槪用人之道, 明賞罰爲貴。 彈劾之事, 或出於風聞, 則爲臺諫者, 當仔細聞見, 然後可以論啓。 且守令, 親民之官, 若頻數遞改, 只貽送迎之弊, 而何能責其成效乎。" 上曰: "刷馬之弊多矣。 各驛無復立之策乎?" <index num="1000556_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "聞<index num="1000557_0" sort="K" type="지명">黃海道</index>刷馬有弊。 頃見<index num="1000558_0" ref="M_0005678" sort="K" type="이름">張雲翼</index>狀啓, 摘出其中奸濫者治之, 故無刷馬之弊云矣。" <index num="1000559_0" ref="M_0002037" sort="K" type="이름">弘微</index>曰: "科擧之事, 最關國家, 而亂離以後, 文敎廢墜, 閭巷之間, 未見挾冊童子, 此極寒心。 頃出童蒙訓導, 責之以敎誨, 則異日作成之化, 猶或可望矣。" <index num="1000560_0" ref="M_0001736" sort="K" type="이름">時獻</index>曰: "國家恢復, 以武備而爲之者不多, 而只在於維持文敎一事而已。 近來學校頹廢, 人不讀書, 振作培養之典, 蔑爾無聞。 自上每下敎於銓曹, 欲得人材, 而人材必待儲養, 然後乃可得也。 今者文籍散缺, 不可盡備, 不得已爲先措備者, <index num="1000561_0" type="서명">《小學》</index>書也。" 上曰: "其板有之乎? 下書求之, 可得以印之乎? 如欲印出, 必有匠人, 備料爲難。" </p>'''

pl = len(html.fromstring(p).xpath('//text()'))
pl

109

51.0

In [198]:
[html.fromstring(p).xpath("index-of(index[@ref="+i+"])") for i in html.fromstring(p).xpath('index/@ref')]

XPathEvalError: Unregistered function

In [199]:
p

'<p>○乙丑/上御別殿。 領事<index num="1000512_0" ref="M_0003630" sort="K" type="이름">柳成龍</index>、知事<index num="1000513_0" ref="M_0061334" sort="K" type="이름">洪進</index>、特進官<index num="1000514_0" ref="M_0049182" sort="K" type="이름">申磼</index>ㆍ宋言愼、參贊官<index num="1000515_0" ref="M_0006601" sort="K" type="이름">洪世恭</index>ㆍ<index num="1000516_0" ref="M_0005431" sort="K" type="이름">李好閔</index>、執義<index num="1000517_0" ref="M_0001736" sort="K" type="이름">金時獻</index>、司諫<index num="1000518_0" ref="M_0002037" sort="K" type="이름">金弘微</index>、檢討官<index num="1000519_0" sort="K" type="이름">鄭㷤</index>、記事官<index num="1000520_0" ref="M_0056678" sort="K" type="이름">趙濈</index>ㆍ<index num="1000521_0" ref="M_0000861" sort="K" type="이름">張晩</index>ㆍ<index num="1000522_0" ref="M_0003584" sort="K" type="이름">柳慶宗</index>入侍, 上講<index num="1000523_0" sort="K" type="이름">周易</index>。 <index num="1000524_0" ref="M_0003630" sort="K" type="이름">成龍</index>曰: "自上欲爲展謁太廟, 人皆以爲感激, 但列聖座次, 當初倉卒不能講定。 頃者<index num="1000525_0" ref="M_0061334" sort="

In [203]:
[i for i in html.fromstring(p).xpath('//text()')]

['○乙丑/上御別殿。 領事',
 '柳成龍',
 '、知事',
 '洪進',
 '、特進官',
 '申磼',
 'ㆍ宋言愼、參贊官',
 '洪世恭',
 'ㆍ',
 '李好閔',
 '、執義',
 '金時獻',
 '、司諫',
 '金弘微',
 '、檢討官',
 '鄭㷤',
 '、記事官',
 '趙濈',
 'ㆍ',
 '張晩',
 'ㆍ',
 '柳慶宗',
 '入侍, 上講',
 '周易',
 '。 ',
 '成龍',
 '曰: "自上欲爲展謁太廟, 人皆以爲感激, 但列聖座次, 當初倉卒不能講定。 頃者',
 '洪進',
 '奉審時, 入而見之, 最爲未安。 不得已先定座次, 然後親祭可矣。" ',
 '洪進',
 '曰: "臣與',
 '成龍',
 '奉審, 以圖形進入矣。 大槪十室, 以西爲上, 而其室之制不長, 不能一行而爲座次, 故回坐於越邊, 所見未安故敢達。" 上曰: "禮曹與大臣議爲。" ',
 '洪進',
 '曰: "小臣之意, 以爲如昭穆之制可也。 連作一室於其外, 則可坐如昭穆。" ',
 '成龍',
 '曰: "今者日寒, 親祭前, 恐不可爲也。" 上曰: "今此',
 '倭賊',
 '不退, 不擊則我國不可保矣。 設使',
 '倭',
 '雖撤退, 後必更來, 我國人情, 謂其一去, 則不復出來云, 可笑。 明春兇謀, 不可測矣。" 上曰: "',
 '孫經略',
 '爲我國題本云, 卿見之乎?" ',
 '成龍',
 '曰: "不見矣。" 上曰: "大臣不得見乎?" ',
 '成龍',
 '曰: "小臣連有病故, 不得來見矣。" 上曰: "承旨',
 '                 ',
 '【',
 '洪世恭',
 '。】',
 '               ',
 '亦見乎?" ',
 '世恭',
 '曰: "暫見矣。" 上曰: "其意以爲, 中原、我國, 形勢相倚, ',
 '倭賊',
 '之來, 若先保',
 '朝鮮',
 ', 則中原可無憂, 而',
 '中',
 '朝若盡力防賊, 則',
 '朝鮮',
 '亦必有德色矣。 軍糧不足, 則',
 '金',
 '、',
 '覆',
 '、',
 '海',
 '、',
 '蓋',
 '               ',
 '                 ',
 '【四州

In [240]:
pr = ''
tglist = re.findall('<index.*?>.+?</index>',p)
nmlist = re.findall('<index.*?>(.+?)</index>',p)
for i, j in enumerate(html.fromstring(p).xpath('//text()')):
    if j in nmlist:
        s = tglist[nmlist.index(j)]
        tglist.pop(nmlist.index(j))
        nmlist.pop(nmlist.index(j))
    else:
        s = '<c>'+j+'</c>'
    pr += s

In [241]:
pr

'<c>○乙丑/上御別殿。 領事</c><index num="1000512_0" ref="M_0003630" sort="K" type="이름">柳成龍</index><c>、知事</c><index num="1000513_0" ref="M_0061334" sort="K" type="이름">洪進</index><c>、特進官</c><index num="1000514_0" ref="M_0049182" sort="K" type="이름">申磼</index><c>ㆍ宋言愼、參贊官</c><index num="1000515_0" ref="M_0006601" sort="K" type="이름">洪世恭</index><c>ㆍ</c><index num="1000516_0" ref="M_0005431" sort="K" type="이름">李好閔</index><c>、執義</c><index num="1000517_0" ref="M_0001736" sort="K" type="이름">金時獻</index><c>、司諫</c><index num="1000518_0" ref="M_0002037" sort="K" type="이름">金弘微</index><c>、檢討官</c><index num="1000519_0" sort="K" type="이름">鄭㷤</index><c>、記事官</c><index num="1000520_0" ref="M_0056678" sort="K" type="이름">趙濈</index><c>ㆍ</c><index num="1000521_0" ref="M_0000861" sort="K" type="이름">張晩</index><c>ㆍ</c><index num="1000522_0" ref="M_0003584" sort="K" type="이름">柳慶宗</index><c>入侍, 上講</c><index num="1000523_0" sort="K" type="이름">周易</index><c>。 </c><index num="1000524_0" ref="M_0003630" sort="K" type="이름">成龍</inde

In [228]:
t = ''
for i in p
    print(i.text)

柳成龍
洪進
申磼
洪世恭
李好閔
金時獻
金弘微
鄭㷤
趙濈
張晩
柳慶宗
周易
成龍
洪進
洪進
成龍
洪進
成龍
倭賊
倭
孫經略
成龍
成龍
                 
世恭
倭賊
朝鮮
中
朝鮮
金
覆
海
蓋
                 
平壤
遼東
義州
朝鮮
申磼
倭
好閔
倭
好閔
好閔
成龍
成龍
黃海道
張雲翼
弘微
時獻
《小學》


['柳成龍',
 '洪進',
 '申磼',
 '洪世恭',
 '李好閔',
 '金時獻',
 '金弘微',
 '鄭㷤',
 '趙濈',
 '張晩',
 '柳慶宗',
 '周易',
 '成龍',
 '洪進',
 '洪進',
 '成龍',
 '洪進',
 '成龍',
 '倭賊',
 '倭',
 '孫經略',
 '成龍',
 '成龍',
 '洪世恭',
 '世恭',
 '倭賊',
 '朝鮮',
 '中',
 '朝鮮',
 '金',
 '覆',
 '海',
 '蓋',
 '平壤',
 '遼東',
 '義州',
 '朝鮮',
 '申磼',
 '倭',
 '好閔',
 '倭',
 '好閔',
 '好閔',
 '成龍',
 '成龍',
 '黃海道',
 '張雲翼',
 '弘微',
 '時獻',
 '《小學》']

In [299]:
tokenizer = RegexpTokenizer(r"\w+|<index.+?</index>")
t = tokenizer.tokenize(sillok['paragraph'][13])
t = [i for i in t if i not in specials]
t

['乙丑',
 '上御別殿',
 ' ',
 '領事',
 '<index num="1000512_0" ref="M_0003630" sort="K" type="이름">柳成龍</index>',
 '知事',
 '<index num="1000513_0" ref="M_0061334" sort="K" type="이름">洪進</index>',
 '特進官',
 '<index num="1000514_0" ref="M_0049182" sort="K" type="이름">申磼</index>',
 'ㆍ宋言愼',
 '參贊官',
 '<index num="1000515_0" ref="M_0006601" sort="K" type="이름">洪世恭</index>',
 '<index num="1000516_0" ref="M_0005431" sort="K" type="이름">李好閔</index>',
 '執義',
 '<index num="1000517_0" ref="M_0001736" sort="K" type="이름">金時獻</index>',
 '司諫',
 '<index num="1000518_0" ref="M_0002037" sort="K" type="이름">金弘微</index>',
 '檢討官',
 '<index num="1000519_0" sort="K" type="이름">鄭㷤</index>',
 '記事官',
 '<index num="1000520_0" ref="M_0056678" sort="K" type="이름">趙濈</index>',
 '<index num="1000521_0" ref="M_0000861" sort="K" type="이름">張晩</index>',
 '<index num="1000522_0" ref="M_0003584" sort="K" type="이름">柳慶宗</index>',
 '入侍',
 ' ',
 '上講',
 '<index num="1000523_0" sort="K" type="이름">周易</index>',
 ' ',
 '<index num="1000524_0" ref="M_0

In [5]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    char = chr(
        kor_begin +
        chosung_base * chosung_list.index(chosung) +
        jungsung_base * jungsung_list.index(jungsung) +
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    if not character_is_korean(c):
        return None
    i = ord(c)
    if (jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if (moum_begin <= i <= moum_end):
        return (' ', c, ' ')

    # decomposition rule
    i -= kor_begin
    cho  = i // chosung_base
    jung = ( i - cho * chosung_base ) // jungsung_base 
    jong = ( i - cho * chosung_base - jung * jungsung_base )    
    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
            (jaum_begin <= i <= jaum_end) or
            (moum_begin <= i <= moum_end))
def levenshtein(s1, s2, cost=None, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug=debug)

    if len(s2) == 0:
        return len(s1)

    if cost is None:
        cost = {}

    # changed
    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return cost.get((c1, c2), 1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            # Changed
            substitutions = previous_row[j] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]
def jamo_levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return jamo_levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return levenshtein(decompose(c1), decompose(c2))/3

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            # Changed
            substitutions = previous_row[j] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(['%.3f'%v for v in current_row[1:]])

        previous_row = current_row

    return previous_row[-1]

In [9]:
jamo_levenshtein('테스트','날람미')

2.6666666666666665

In [19]:
gwanjik = pd.read_sql_table('gwanjik',con=engine)

In [22]:
gwanjik['gye'] + gwanjik['sa'] + gwanjik['jik']

0           
1         서얼
2       공릉참봉
3       산학별제
4       수의부위
        ... 
2252      사사
2253     선전관
2254     우사어
2255    승의부위
2256    진무부위
Length: 2257, dtype: object